In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_no2_full.csv

File './../../../dataset/airquality_no2_full.csv' already there; not retrieving.



In [3]:
path = "./../../../dataset/airquality_no2_full.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (7393, 12)


,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S5(O3),PT08.S1(CO),true_label,given_label,true_error
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1268.0,1360.0,0.017948,0.678878,0
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,972.0,1292.0,-0.422958,0.302648,0
2,evening,wednesday,11.9,54.0,0.7502,939.0,1140.0,1074.0,1402.0,0.038944,0.291333,0
3,evening,wednesday,11.0,60.0,0.7867,948.0,1092.0,1203.0,1376.0,0.206908,0.373368,0
4,night,wednesday,11.2,59.6,0.7888,836.0,1205.0,1110.0,1272.0,0.080935,0.107462,0


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S5(O3),PT08.S1(CO),given_label
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1268.0,1360.0,0.678878
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,972.0,1292.0,0.302648


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7393
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1274.97 MB
	Train Data (Original)  Memory Usage: 1.36 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtyp

[1000]	valid_set's l2: 0.0238809	valid_set's r2: 0.896938


	0.9765	 = Validation score   (r2)
	4.49s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ...
	0.9698	 = Validation score   (r2)
	1.73s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
	0.98	 = Validation score   (r2)
	4.19s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.981	 = Validation score   (r2)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 10.8s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("./../trained_models/")
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when u

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.993292,0.976517,0.082097,0.016727,4.487710,0.082097,0.016727,4.487710,1,True,1
1,RandomForestMSE,0.992997,0.969763,0.187515,0.066401,1.730114,0.187515,0.066401,1.730114,1,True,2
2,WeightedEnsemble_L2,0.987612,0.981009,0.158379,0.028988,8.709241,0.027774,0.000499,0.032515,2,True,4
3,NeuralNetFastAI,0.981503,0.979964,0.048508,0.011762,4.189016,0.048508,0.011762,4.189016,1,True,3


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7393
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1343.82 MB
	Train Data (Original)  Memory Usage: 1.36 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

[1000]	valid_set's l2: 0.0252954	valid_set's r2: 0.897898
[1000]	valid_set's l2: 0.0247472	valid_set's r2: 0.892655
[1000]	valid_set's l2: 0.0264818	valid_set's r2: 0.885556
[1000]	valid_set's l2: 0.0244745	valid_set's r2: 0.888657
[2000]	valid_set's l2: 0.0237253	valid_set's r2: 0.889775
[1000]	valid_set's l2: 0.0252198	valid_set's r2: 0.888059
[2000]	valid_set's l2: 0.0241719	valid_set's r2: 0.890251
[3000]	valid_set's l2: 0.0238763	valid_set's r2: 0.890362
[4000]	valid_set's l2: 0.0238284	valid_set's r2: 0.890577
[1000]	valid_set's l2: 0.0254998	valid_set's r2: 0.889416
[2000]	valid_set's l2: 0.0248317	valid_set's r2: 0.889646
[1000]	valid_set's l2: 0.0213319	valid_set's r2: 0.884685
[2000]	valid_set's l2: 0.020827	valid_set's r2: 0.884925
[1000]	valid_set's l2: 0.0238887	valid_set's r2: 0.893101


	0.9758	 = Validation score   (r2)
	27.94s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	0.9692	 = Validation score   (r2)
	1.5s	 = Training   runtime
	0.18s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set numb

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.997031,0.975754,1.564720,0.226605,27.941201,1.564720,0.226605,27.941201,1,True,1
1,RandomForestMSE_BAG_L1,0.995860,0.969238,0.220904,0.181208,1.502783,0.220904,0.181208,1.502783,1,True,2
2,WeightedEnsemble_L2,0.989924,0.980616,1.905766,0.295883,53.914700,0.001620,0.000195,0.057394,2,True,4
3,NeuralNetFastAI_BAG_L1,0.983476,0.979351,0.339427,0.069083,25.916105,0.339427,0.069083,25.916105,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))